In [23]:
from skimage.io import collection, imread
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
import re
import glob

from datetime import datetime
import nltk

# How is the public opinion about a company correlated to it's market value?

A company's market value is variable and depends on a lot of factors. The price is a reflection of the company's perceived value – what the public is willing to pay for a piece of the company. It can and will rise and fall, based on a variety of factors in the global landscape and within the company itself. One of which is becoming more influential than ever - the public's opinion on social media.

To analyze this correlation we'll look at two datasets. The first one contains over 3 million unique tweets with their information such as tweet id, author of the tweet, post date, the text body of the tweet, and the number of comments, likes, and retweets of tweets matched with the related company.

The second one will just have daily stock price records (from the Forbes2000) for us to make a reference with.

### 1. Data Acquisition 

So first let's read the tweets dataset into pandas and inspect a small sample from the two dataframes.

In [24]:
tweets = pd.read_csv('./top-companies-tweets/Tweet.csv')
tweets.sample(5)

,tweet_id,writer,post_date,body,comment_num,retweet_num,like_num
996071,726810103869923328,It_c0nsulting,1462120006,Street View catches #Google's own car speeding...,0,0,0
2313247,976115577210126336,canuck2usa,1521559063,LMAO WTH is this UPDATE: Google Says It's Comm...,5,4,14
1364823,780892454727716864,App_sw_,1475014244,#Microsoft joins Adobe to compete with Amazon ...,0,0,0
1025061,730779379752558593,It_c0nsulting,1463066356,#Google to roll out ''Tap to Translate'' featu...,0,0,0
2094605,923577092159160321,MobilePotpurri,1509032912,Surveys today on whether people will buy auton...,0,2,2


In [25]:
tweets_company = pd.read_csv('./top-companies-tweets/Company_Tweet.csv')
tweets_company.sample(5)

,tweet_id,ticker_symbol
3989255,1091716285900247045,TSLA
1420184,1207776372136955904,AAPL
1058391,990266661239476224,AAPL
4326104,1208964139982491650,TSLA
576346,753721574927142912,AAPL


So next up let's read the datasets for each of the stocks, which we are monitoring. We will save them in a dictionary with the key, being the company's tick name and the value - it's stock prices over time dataset.

In [26]:
stocks_df = {}
for name in glob.glob('./stocks/*'):
    stocks_df[name.split('\\')[-1].split('.')[0]] = pd.read_csv(name)
stocks = pd.concat(stocks_df)
stocks.sample(10)

Date         Low        Open     Volume        High  \
AAPL 5757  02-10-2003    0.362143    0.371429  204058400    0.371429   
GOOG 1394  04-03-2010  272.080170  272.229614    6391467  277.026642   
AMZN 3000  20-04-2009   76.830002   78.440002    8686900   79.790001   
AAPL 44    17-02-1981    0.116629    0.116629   12275200    0.117188   
     8453  19-06-2014   22.834999   23.072500  142112000   23.075001   
AMZN 3806  28-06-2012  218.750000  223.919998    2994800  224.619995   
     4163  29-11-2013  388.619995  389.100006    2406000  394.100006   
MSFT 6832  18-04-2013   28.500000   28.950001   56906600   28.980000   
GOOG 3298  25-09-2017  909.700012  925.450012    1856800  926.400024   
TSLA 1227  14-05-2015   48.250000   48.964001   14479500   48.978001   

                Close  Adjusted Close  
AAPL 5757    0.367321        0.315860  
GOOG 1394  276.259521      276.259521  
AMZN 3000   77.570000       77.570000  
AAPL 44      0.116629        0.091552  
     8453   22.965000       20.660538  
AMZN 3806  221.309998      221.309998  
     4163  393.619995      393.619995  
MSFT 6832   28.790001       24.250801  
GOOG 3298  920.969971      920.969971  
TSLA 1227   48.820000       48.820000

### 2. Data Tidying and Cleaning

First let's combine the two tables from the twitter dataset, convert the dates to a datetime object and rename the column.

In [27]:
tweets = pd.merge(tweets, tweets_company, on = "tweet_id")

In [28]:
tweets["date"] = pd.to_datetime(tweets.post_date, unit='s')
tweets = tweets.drop(columns="post_date")
tweets.sample(5)


,tweet_id,writer,body,comment_num,retweet_num,like_num,ticker_symbol,date
808990,676084306821844992,ACInvestorBlog,"$AMZN If prices continue to head lower, the ri...",0,0,4,AMZN,2015-12-13 17:00:34
2736604,986262912229572609,RobSkalawag,Trust in Facebook has spectacularly nosedived ...,0,0,0,AMZN,2018-04-17 15:19:36
1787919,815989218845462528,App_sw_,Rumor: Samsung Galaxy S8 to have a #Microsoft ...,0,0,0,MSFT,2017-01-02 18:32:44
1407441,763389533853405184,OptiontradinIQ,What Are Market Makers - $SPY $SPX $RUT $AAPL ...,0,1,2,AAPL,2016-08-10 15:00:22
1289868,745689078427947008,PortfolioBuzz,Track trending assets in 1 watchlist #fintech ...,0,0,0,AAPL,2016-06-22 18:45:05


We will see what timeframe does our dataset cover, by getting the data of the earliest and latest tweets.

In [29]:
tweets.date.min(), tweets.date.max()

(Timestamp('2015-01-01 00:00:57'), Timestamp('2019-12-31 23:55:53'))

So it has data from 01.01.2015 to 31.12.2019, so basically from 2015 to the beginning of 2020. Knowing this we can filter out the stock prices to be only in this period of time. But first we have to covert the "Date" column to datetime.

In [30]:
def string_to_date(date_string):
    return datetime.strptime(date_string, "%d-%m-%Y")
stocks.Date = pd.to_datetime(stocks.Date.apply(string_to_date))

In [31]:
stocks = stocks[(stocks.Date >= '01-01-2015') & (stocks.Date < '01-01-2020')]
stocks.sample(10)

Date          Low         Open     Volume         High  \
GOOG 3729 2019-06-13  1080.150024  1083.640015    1057700  1094.170044   
     2707 2015-05-21   535.979980   537.950012    1462700   543.840027   
AAPL 9421 2018-04-24    40.305000    41.417500  134768000    41.582500   
AMZN 5650 2019-10-28  1742.500000  1748.060059    3708900  1778.699951   
     4769 2016-04-28   599.200012   615.539978    7872600   626.799988   
GOOG 3205 2017-05-12   927.849976   931.530029    1050600   933.440002   
AMZN 4475 2015-02-27   379.790009   384.000000    2532300   385.989990   
     4936 2016-12-23   757.989990   764.549988    1981600   766.500000   
MSFT 7807 2017-03-02    63.880001    64.690002   24539600    64.750000   
     8405 2019-07-18   134.669998   135.550003   30381800   136.619995   

                 Close  Adjusted Close  
GOOG 3729  1088.770020     1088.770020  
     2707   542.510010      542.510010  
AAPL 9421    40.735001       39.234344  
AMZN 5650  1777.079956     1777.079956  
     4769   602.000000      602.000000  
GOOG 3205   932.219971      932.219971  
AMZN 4475   380.160004      380.160004  
     4936   760.590027      760.590027  
MSFT 7807    64.010002       60.100285  
     8405   136.419998      133.469391

As we can see we don't have data for every day, because the stock market funcitons only on workdays, unlike twitter.  We will find a way to work around this later.

Now lets strip the data down to just one column - the value, which we will calculate by getting the mean of the Open and Close prices. 

In [32]:
stocks["Value"] = (stocks.Open + stocks.Close) / 2
stocks = stocks.drop(columns=['Low', 'Open', 'High', 'Close', 'Adjusted Close'])
stocks.sample(10)

Date    Volume        Value
TSLA 1475 2016-05-09  23882000    42.464001
GOOG 3185 2017-04-13   1122400   822.850006
     3641 2019-02-06   2105600  1127.399963
MSFT 7397 2015-07-16  26271700    46.334999
     8475 2019-10-25  25959700   140.034996
     8230 2018-11-02  37680200   106.320004
     8076 2018-03-27  56569000    92.205002
TSLA 1201 2015-04-08  31515500    41.587000
     1454 2016-04-08  36819500    51.056999
MSFT 7308 2015-03-10  39159700    42.189999

It is a little inconvenient to have the stock name as an index instead of it being a regular column. We will fix that and also change the column names to match the twitter dataset.

In [33]:
stocks = stocks.reset_index(level=0)

In [34]:
stocks.columns = ["ticker_symbol", "date", "volume", "value"]
stocks.sample(10)
stocks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6290 entries, 8589 to 2393
Data columns (total 4 columns):
ticker_symbol    6290 non-null object
date             6290 non-null datetime64[ns]
volume           6290 non-null int64
value            6290 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(1), object(1)
memory usage: 245.7+ KB


 So next up lets take a look at the datatypes and null values for the twitter dataset.

In [35]:
tweets.info(null_counts = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4336445 entries, 0 to 4336444
Data columns (total 8 columns):
tweet_id         4336445 non-null int64
writer           4280526 non-null object
body             4336445 non-null object
comment_num      4336445 non-null int64
retweet_num      4336445 non-null int64
like_num         4336445 non-null int64
ticker_symbol    4336445 non-null object
date             4336445 non-null datetime64[ns]
dtypes: datetime64[ns](1), int64(4), object(3)
memory usage: 297.8+ MB


Everything looks good, except the ticker_symbol which should be a category. Also the writer column has quite a few missing records, but we won't be using it for our model and analysis, so we can discard it altogether.

In [36]:
tweets.ticker_symbol = tweets.ticker_symbol.astype('category')
tweets = tweets.drop(columns=["writer"])
tweets.head(7)

,tweet_id,body,comment_num,retweet_num,like_num,ticker_symbol,date
0,550441509175443456,"lx21 made $10,008 on $AAPL -Check it out! htt...",0,0,1,AAPL,2015-01-01 00:00:57
1,550441672312512512,Insanity of today weirdo massive selling. $aap...,0,0,0,AAPL,2015-01-01 00:01:36
2,550441732014223360,S&P100 #Stocks Performance $HD $LOW $SBUX $TGT...,0,0,0,AMZN,2015-01-01 00:01:50
3,550442977802207232,$GM $TSLA: Volkswagen Pushes 2014 Record Recal...,0,0,1,TSLA,2015-01-01 00:06:47
4,550443807834402816,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,2015-01-01 00:10:05
5,550443807834402816,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,TSLA,2015-01-01 00:10:05
6,550443808606126081,Swing Trading: Up To 8.91% Return In 14 Days h...,0,0,1,AAPL,2015-01-01 00:10:05


As we can see there seem to be a lot of duplicate bodies in our dataset. We want to remove them and this is exactly what the following code does. 

In [37]:
tweets = tweets.drop_duplicates(subset=["body"])

### 3. Text Preparation and Exploration

Before we start working with the text, we have to prepare it and take a quick look at some statistics about it. First let's convert all the tweets' bodies into lowercase.

In [38]:
tweets.body = tweets.body.str.lower()

NLTK provides a small corpus of stop words that we will load into a list, based on which we'll later filter them out from the tweets.

In [39]:
stopwords = nltk.corpus.stopwords.words("english")
stopwords.append("")

Now let's split the text into single words and remove all the stopwords from it.

In [40]:
def string_into_words(str): 
    return [w for w in re.split("\W+", str) if w not in stopwords]
tweets.body = tweets.body.apply(string_into_words)
tweets.sample(5)

,tweet_id,body,comment_num,retweet_num,like_num,ticker_symbol,date
1329344,752886169873084416,"[guess, tsla, top, secret, project, solar, pow...",0,0,0,TSLA,2016-07-12 15:23:45
4008734,1157278373065846784,"[much, board, member, larry, ellison, pay, per...",0,0,1,TSLA,2019-08-02 13:13:59
118703,568812310078365696,"[sreeiyer1, best, evidence, aapl, working, ele...",0,0,0,AAPL,2015-02-20 16:39:57
1347261,755739019145650178,"[apple, less, 10, iphone, owners, plan, upgrad...",0,0,0,AAPL,2016-07-20 12:19:57
638827,645458270887395328,"[weekly, chart, analysis, aapl, http, dlvr, cc...",0,0,0,AAPL,2015-09-20 04:43:38


Now we can look at the frequency distribution of the words. This will essentilaly 